## Calculate ice volume from Farinotti et al. (2019) for the Columbia Basin
### This is year 2000 ice volume as it is based on the SRTM

In [1]:
import matplotlib.pyplot as plt
from matplotlib import pylab
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.ticker as ticker
from osgeo import gdal
import salem
from salem import *

import fiona, rasterio
import geopandas as gpd
from rasterio.plot import show
import rasterio.plot as rplt
from rasterstats import zonal_stats

In [2]:
def loadshp_as_gpd(shp):
    data_shp = gpd.read_file(shp)
    return data_shp

In [3]:
path = '/home/pelto/GIS/GPR/ComparisonData/RGI60-02/'

##list of all glaciers in the Basin
basin = '/home/pelto/GIS/GLIMS_BC/glims_all/rgi_60_UTM_CB.csv' 
#'/home/pelto/GIS/GLIMS_BC/glims_all/02_rgi60_WesternCanadaUS_UTM_CB.shp'
basin = '/home/pelto/GIS/GLIMS_BC/glims_all/02_rgi60_Columbia_mtns.csv'
basin = pd.read_csv(basin)
basin

,RGIId,GLIMSId,BgnDate,EndDate,CenLon,CenLat,O1Region,O2Region,Area,Zmin,...,Slope,Aspect,Lmax,Status,Connect,Form,TermType,Surging,Linkages,Name
0,RGI60-02.00121,G242878E49728N,20049999,20069999,-117.12200,49.72810,2,3,0.184,2428,...,28.1,67,561,0,0,0,0,0,9,NaN
1,RGI60-02.00145,G242840E49757N,20049999,20069999,-117.16000,49.75670,2,3,0.531,2456,...,17.0,348,1258,0,0,0,0,0,9,NaN
2,RGI60-02.00146,G242849E49755N,20049999,20069999,-117.15100,49.75460,2,3,0.375,2486,...,19.0,43,831,0,0,0,0,0,9,NaN
3,RGI60-02.00147,G242863E49749N,20049999,20069999,-117.13700,49.74900,2,3,1.801,2258,...,19.2,33,1738,0,0,0,0,0,9,NaN
4,RGI60-02.00158,G242887E49766N,20049999,20069999,-117.11300,49.76560,2,3,0.252,2471,...,18.0,42,780,0,0,0,0,0,9,NaN
5,RGI60-02.00184,G242340E49785N,20049999,20069999,-117.66000,49.78520,2,3,0.081,2414,...,29.3,314,335,0,0,0,0,0,9,NaN
6,RGI60-02.00185,G242874E49784N,20049999,20069999,-117.12600,49.78380,2,3,0.309,2346,...,25.0,10,709,0,0,0,0,0,9,NaN
7,RGI60-02.00186,G242368E49789N,20049999,20069999,-117.63200,49.78890,2,3,0.077,2326,...,28.2,340,352,0,0,0,0,0,9,NaN
8,RGI60-02.00222,G243631E49820N,20049999,20069999,-116.36900,49.82000,2,3,0.073,2514,...,21.6,14,474,0,0,0,0,0,9,NaN
9,RGI60-02.00227,G242259E49830N,20049999,20069999,-117.74100,49.82990,2,3,0.225,2380,...,24.1,59,766,0,0,0,0,0,9,NaN


In [4]:
path+basin.RGIId.unique()[0]+'_thickness.tif'

'/home/pelto/GIS/GPR/ComparisonData/RGI60-02/RGI60-02.00121_thickness.tif'

In [5]:
V= []
for n in basin.RGIId.unique():
    f=salem.open_xr_dataset(path+n+'_thickness.tif')
    v = f.data * 25.**2
    V.append(np.sum(v))
   

In [6]:
Volume = np.sum(V) /1e9
print('Columbia Basin Ice volume year 2000 (Farinotti et al. 2019):', np.round(Volume,2), 'km3')

Columbia Basin Ice volume, Farinotti 2019: 105.54 km3


In [7]:
A = basin.Area.sum()
C = len(basin.RGIId)
print('Area of glaciers in the Basin in 2005:', np.round(A,2), 'km2,', 'number of glaciers in the Basin:', C)

Area of glaciers in the Basin in 2005: 1956.57 km2, number of glaciers in the Basin: 2308
